# Exploratory Data Analysis

#### Global variables

##### In order to streamline the pipeline there are several data variables as well as libraries that we will work with over multiple instances, therefore, it is best to isolate them and declare them globally at the start of the file for ease of access.

In [17]:
#@authors: Michelle Collins, Jack Eaton Kilgallen
import mne
import os
import pandas as pd
import numpy as np

mne.set_log_level('CRITICAL')

folder_loc = r"C:/Users/Bogdan/Documents/College/4th Year/FYP-main/EEG_data_Analysis/Data/"

subjects = ["11", "13", "14", "16", "17", "19", "20", "22", "24", "25", "26","28", "29", "31", "32", "33", "34", "52", "53", "55", "56", "58", "59", "60", "61"]

trials = ["1", "2"]

tasks = ["VPA","SG","VS"]

controls = ["11", "13", "14", "16", "17", "19", "20", "22", "24", "25", "26","28", "29", "31", "32", "33", "34"]

patients = ["52", "55", "56", "58", "59", "60", "61"]

VPApicks = ["PO3", "PO4", "Pz"]

to_be_repaired = ["11", "13", "14", "16", "17", "19", "20", "22", "24", "25", "26"]

#### Renaming
##### One of the first steps towards handling the data is renaming it to ensure anonimity of the patients and the removal of bias, here this python script takes in the original data file and relabels it removing the name for a designated number and only keeping the experiment type

In [15]:
def rename_raw_data():

    for folder in os.listdir(folder_loc):
        for file in os.listdir(folder_loc + folder):
            
            subject = folder
            
                
            if "_1_SG".lower() in file.lower() or \
            "_1_VS".lower() in file.lower() or \
            "_1_VPA".lower() in file.lower() or \
            "_2_SG".lower() in file.lower() or \
            "_2_VS".lower() in file.lower() or \
            "_2_VPA".lower() in file.lower():
                print("File " + file + " already exists")
                continue

            if ".bdf".lower() not in file.lower():
                continue
                
            #This takes each file and assigns its trial as 1 or 2,
            # depending on whether or not the string "Exp2b" is in the original name
            if "Exp2b".lower() in file.lower():
                trial = "2"
            else:
                trial = "1"
                
            #identifies which task it is
            if " VPA".lower() in file.lower():
                task = "VPA"
            elif "Spatial".lower() in file.lower():
                task = "SG"
            elif "Visual Search".lower() in file.lower():
                task = "VS"
            elif "finger".lower() in file.lower():
                continue
            elif "eyes".lower() in file.lower():
                continue
            
            #flags files which may not fall into original naming convention
            else:
                print("Did not find task for " + file)
                continue
        
            old_filename = os.path.join(folder_loc, folder, file)
            new_filename = rf"C:\Users\Bogdan\Documents\College\4th Year\FYP-main\EEG_data_Analysis\Data\{subject}\{subject}_{trial}_{task}.bdf"
            print("Renaming " + old_filename + " to: " + new_filename )
            
            os.rename(old_filename, new_filename)
            
print("finished renaming")

finished renaming


#### Preprocessing
##### The next step is to preprocess the data, there are parts of the raw input that need filtering out, such as eye movement, and this next segment of code handles that.

In [4]:
def set_meta_data(raw):
    raw.set_channel_types({'EXG1': 'misc','EXG2':'misc','EXG3':'misc','EXG4':'misc', 'EXG5':'eog','EXG6': 'eog','EXG7':'misc','EXG8':'misc'})
    try:
        raw.drop_channels(['GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp'])
    except:
        pass
    
    raw.set_montage('standard_1020')
    
def downsample(raw):
    current_sfreq = raw.info['sfreq']
    desired_sfreq = 256
    decim = np.round(current_sfreq / desired_sfreq).astype(int)
    obtained_sfreq = current_sfreq / decim
    lowpass_freq = obtained_sfreq / 3.
    raw.filter(l_freq=None, h_freq=lowpass_freq)
    
def remove_ocular_noise(raw):
    ica = mne.preprocessing.ICA(n_components=0.95, random_state=97)
    ica.fit(raw)
    ica.exclude = []
    eog_indices, eog_scores = ica.find_bads_eog(raw)
    ica.exclude = eog_indices
    ica.apply(raw)
    
def preprocessing(raw):
    set_meta_data(raw)
    downsample(raw)
    raw.filter(l_freq=0.5, h_freq=None)
    raw.notch_filter(freqs=[50])
    remove_ocular_noise(raw)
    return raw


#### Repair event data

In [7]:
def repair_event_data(subject, trial, task, raw):
    event_file_name = "{}_{}_{}_(Corrected).evt".format(subject, trial, task)
    event_file_loc = os.path.join(folder_loc, str(subject), event_file_name)
    events_as_df = pd.read_csv(event_file_loc, delimiter="\t", index_col = False)
    
    rows_to_drop = []
    for row_index in range(len(events_as_df)):
    # If trigger number is 41, or unexpectedly long (i.e a date string), or time data is recorded incorrectly (negative time) drop this row.
        if str(events_as_df.loc[row_index, "TriNo"]) == "41" or \
            len(str(events_as_df.loc[row_index, "TriNo"])) > 5 or \
            int(events_as_df.loc[row_index, 'Tmu ']) < 0:
            rows_to_drop.append(row_index)
        
    events_as_df = events_as_df.drop(events_as_df.columns[3], axis=1).drop(rows_to_drop)
    events_as_df = events_as_df[events_as_df['TriNo'] != '-']
    events_as_df['TriNo'] = pd.to_numeric(events_as_df['TriNo'])
    events_as_df['Code'] = events_as_df['Code'].fillna(0)
    events_as_df['Tmu '] = events_as_df['Tmu '] / 1953.003030082584
    events_as_df = events_as_df.loc[:, ~events_as_df.columns.str.contains("^Unnamed")]
    
    event_file_name = "{}_{}_{}.eve".format(subject, trial, task)
    event_file_loc = os.path.join(folder_loc, str(subject), event_file_name)
    np.savetxt(event_file_loc, events_as_df.values.astype(int), delimiter='\t')
    
    events = mne.read_events(event_file_loc)
    for x in range(len(events)):
        if events[x][2] > 1 and events[x][2] < 10:
            events[x][2] -= 2
    raw.add_events(events, stim_channel='Status', replace=True)
    return raw

#### Event data processing

In [8]:
def relabel_VPA_events(events):
    for x in range(len(events) - 1):
        #F/C/C
        if events[x][2] == 4 and events[x+1][2] == 10:
            events[x][2] = 12
        #F/C/I
        elif events[x][2] == 4 and events[x+1][2] == 11:
            events[x][2] = 13
        #F/I/C
        elif events[x][2] == 5 and events[x+1][2] == 10:
            events[x][2] = 14
        #F/I/I
        elif events[x][2] == 5 and events[x+1][2] == 11:
            events[x][2] = 15
        #T/C/C
        elif events[x][2] == 6 and events[x+1][2] == 10:
            events[x][2] = 16
        #T/C/I
        elif events[x][2] == 6 and events[x+1][2] == 11:
            events[x][2] = 17
        #T/I/C
        elif events[x][2] == 7 and events[x+1][2] == 10:
            events[x][2] = 18
        #T/I/I
        elif events[x][2] == 7 and events[x+1][2] == 11:
            events[x][2] = 19

def relabel_VS_events(events):
    for x in range(len(events) - 1):
        #FLEFT/C
        if events[x][2] == 1 and events[x+1][2] == 10:
            events[x][2] = 12
        #FLEFT/I
        elif events[x][2] == 1 and events[x+1][2] == 11:
            events[x][2] = 13
        #LEFT/C
        elif events[x][2] == 2 and events[x+1][2] == 10:
            events[x][2] = 14
        #LEFT/I
        elif events[x][2] == 2 and events[x+1][2] == 11:
            events[x][2] = 15
        #RIGHT/C
        elif events[x][2] == 3 and events[x+1][2] == 10:
            events[x][2] = 16
        #RIGHT/I
        elif events[x][2] == 3 and events[x+1][2] == 11:
            events[x][2] = 17
        #FRIGHT/C
        elif events[x][2] == 4 and events[x+1][2] == 10:
            events[x][2] = 18
        #FRIGHT/I
        elif events[x][2] == 4 and events[x+1][2] == 11:
            events[x][2] = 19

def relabel_SG_events(events):
    for x in range(len(events) - 1):
        #NRSL/C
        if events[x][2] == 1 and events[x+1][2] == 10:
            events[x][2] = 12
        #NRSL/I
        elif events[x][2] == 1 and events[x+1][2] == 11:
            events[x][2] = 13
        #RSL/C
        elif events[x][2] == 2 or events[x][2] == 3 or events[x][2] == 4 and events[x+1][2] == 10:
            events[x][2] = 14
        #RSL/I
        elif events[x][2] == 2 or events[x][2] == 3 or events[x][2] == 4 and events[x+1][2] == 11:
            events[x][2] = 15
        #NRNL/C
        elif events[x][2] == 5 and events[x+1][2] == 10:
            events[x][2] = 16
        #NRNL/I
        elif events[x][2] == 5 and events[x+1][2] == 11:
            events[x][2] = 17
        #RNL/C
        elif events[x][2] == 6 or events[x][2] == 7 or events[x][2] == 8 and events[x+1][2] == 10:
            events[x][2] = 18
        #RNL/I
        elif events[x][2] == 6 or events[x][2] == 7 or events[x][2] == 8 and events[x+1][2] == 11:
            events[x][2] = 19

def event_data_processing(raw, task):
    events = mne.find_events(raw, stim_channel='Status')
    if task == "VPA":
        relabel_VPA_events(events)
    elif task == "VS":
        relabel_VS_events(events)
    elif task == "SG":
        relabel_SG_events(events)
        
    raw.add_events(events, stim_channel='Status', replace=True)
    return raw

#### Main function
##### This is the main body of the pipeline where all previous segments get called and execute their processes.

In [19]:
#@authors: Michelle Collins
rename_raw_data()

for subject in subjects:
    for trial in trials:
        for task in tasks:
            print("Now pipelining subject: " + subject + ", trial: " + trial + ", task: " + task)
            file_name = "{}_{}_{}.bdf".format(subject, trial, task)
            file_loc = os.path.join(folder_loc, subject, file_name)
            
            raw = mne.io.read_raw_bdf(file_loc, preload=True)
            raw = preprocessing(raw)
            
            if subject in to_be_repaired:
                try:
                    raw = repair_event_data(subject, trial, task, raw)
                except FileNotFoundError as e:
                    print(e)
                    
            raw = event_data_processing(raw, task)
            file_name_cleaned = "{}_{}_{}_(Cleaned).fif".format(subject, trial, task)
            file_loc = os.path.join(folder_loc, subject, file_name_cleaned)
            raw.save(file_loc, overwrite=True)
            print("Cleaned file " + file_name_cleaned + " saved")

File 11_1_SG.bdf already exists
File 11_1_VPA.bdf already exists
File 11_1_VS.bdf already exists
File 11_2_SG.bdf already exists
File 11_2_VPA.bdf already exists
File 11_2_VS.bdf already exists
File 12_1_SG.bdf already exists
File 12_1_VPA.bdf already exists
File 12_1_VS.bdf already exists
File 12_2_SG.bdf already exists
File 12_2_VPA.bdf already exists
File 12_2_VS.bdf already exists
File 13_1_SG.bdf already exists
File 13_1_VPA.bdf already exists
File 13_1_VS.bdf already exists
File 13_2_SG.bdf already exists
File 13_2_VPA.bdf already exists
File 13_2_VS.bdf already exists
File 14_1_SG.bdf already exists
File 14_1_VPA.bdf already exists
File 14_1_VS.bdf already exists
File 14_2_SG.bdf already exists
File 14_2_VPA.bdf already exists
File 14_2_VS.bdf already exists
File 15_2_SG.bdf already exists
File 15_2_VPA.bdf already exists
File 15_2_VS.bdf already exists
Renaming C:/Users/Bogdan/Documents/College/4th Year/FYP-main/EEG_data_Analysis/Data/15\Marie McEneaney 15 Spatial Grid Task.bdf

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Bogdan\\Documents\\College\\4th Year\\FYP-main\\EEG_data_Analysis\\Data\\22\\22_1_VPA.bdf'